# Oil Exploration

**Author:** Patricio Ortiz

**Context:** 
We will pre process the data of oil essays. The data will be extracted from static files. Those static files are stored on raw folder, separated by its origin. On one hand, we have data from client EMIN that takes his oil data from ALS while in the other I have data from client CDA that takes the data from Finning Platform.

The core idea of this notebook is to be able to process the raw data from each client and be able to generate a Pipeline that is able to generate comments based on LLM (OpenAI).

ALS Data has one key division to be taken care of : A machine (equipment_tag) has more than one component (compartment_name) and each component may have 1 or more reports.
One report is equivalent to one sample (one row in the available data) and one report is build based on multiple essays.

Our work is to explore the data, clean it for AI consumption, compute limits that triggers alerts (one alert is assigned to 1 report based on essay data), generate the AI comments and finally be able to visualize all the key data to the client.


## Librería

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore", message="Workbook contains no default style")

os.chdir('../data/oil')  # Cambiar al directorio raíz del proyecto

## Auxiliar

We have created an auxiliar dataframe that help us map the chemical essays to the same name/structure.

In [2]:
essays = pd.read_excel('essays_elements.xlsx', engine='openpyxl')
essays

,Element,ElementNameSpanish,ElementNameEnglish,GroupElement
0,Fe,Hierro,Iron,Desgaste
1,Cr,Cromo,Chromium,Desgaste
2,Al,Aluminio,Aluminum,Desgaste
3,Cu,Cobre,Copper,Desgaste
4,Pb,Plomo,Lead,Desgaste
5,Ni,Níquel,Nickel,Desgaste
6,Ag,Plata,Silver,Desgaste
7,Sn,Estaño,Tin,Desgaste
8,Ti,Titanio,Titanium,Desgaste
9,V,Vanadio,Vanadium,Desgaste


Since the file is incomplete we will use only the rows without null values.

In [3]:
# drop essays not fully mapped
essays.dropna(inplace=True)
essays

,Element,ElementNameSpanish,ElementNameEnglish,GroupElement
0,Fe,Hierro,Iron,Desgaste
1,Cr,Cromo,Chromium,Desgaste
2,Al,Aluminio,Aluminum,Desgaste
3,Cu,Cobre,Copper,Desgaste
4,Pb,Plomo,Lead,Desgaste
5,Ni,Níquel,Nickel,Desgaste
6,Ag,Plata,Silver,Desgaste
7,Sn,Estaño,Tin,Desgaste
8,Ti,Titanio,Titanium,Desgaste
9,V,Vanadio,Vanadium,Desgaste


## CDA

Data of CDA comes from Finning Lab and it is stored on "data/oil/raw/cda" separated on excel files by id of the machine.

In [4]:
raw_cda_folder = 'raw/cda'
processed_cda_folder = 'processed/cda'

df_cda = pd.concat([pd.read_excel(os.path.join(raw_cda_folder, file),
                               engine='openpyxl') for file in os.listdir(raw_cda_folder) if file.endswith('.xlsx')],
                ignore_index=True)

df_cda.dropna(how='all', inplace=True)
df_cda.head()

,Estado,No. de control de laboratorio,Fecha de laboratorio,Fecha de Toma de Muestra,Model,ID de equipo,No. de serie del equipo,Compartimento,Horas,Meter on Fluid,...,White Metal,Dust,Organic Material,Polymers,Fibers,Gel/Mud,Beads,Others,FL Pt,FL Pt (°F)
1,No se requiere acción,R120-56014-0277,2026-01-14,2026-01-07,789C,T-09,2BW01106,MANDO FINAL TRASERO IZQUIERDO,112380.0,5100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,No se requiere acción,R120-56014-0279,2026-01-14,2026-01-07,789C,T-09,2BW01106,TRANSMISION,112380.0,477.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,No se requiere acción,R120-56014-0280,2026-01-14,2026-01-07,789C,T-09,2BW01106,CONVERTIDOR,112380.0,2037.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,No se requiere acción,R120-56014-0278,2026-01-14,2026-01-07,789C,T-09,2BW01106,MANDO FINAL TRASERO DERECHO,112380.0,5100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,No se requiere acción,R120-56014-0282,2026-01-14,2026-01-07,789C,T-09,2BW01106,SISTEMA HIDRAULICO,112380.0,2037.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Each Column with the values of the essays are well named
> columns = sample | elementX | elementY | ...

Therefore, we only need to map them onto a common pattern and extract the key columns.

In [5]:
# Rename main columns
df_cda.rename(columns={'ID de equipo' : 'unitId',
                   'No. de control de laboratorio': 'sampleNumber',
                   'No. de serie del equipo' : 'machineSerialNumber',
                   'Component Serial Number' : 'componentSerialNumber',
                   'Compartimento' : 'componentName',
                   'Model' : 'machineModel',
                   'Horas' : 'machineHours',
                   'Component Meter' : 'componentHours',
                   'Meter on Fluid' : 'oilMeter',
                   'Fluid Brand' : 'oilBrand',
                   'Fluid Type' : 'oilType',
                   'Fluid Weight' : 'oilWeight',
                   'Fecha de Toma de Muestra' : 'sampleDate'}, inplace=True)

# Rename essay columns
essays_mapping = dict(zip(essays['Element'], essays['ElementNameSpanish']))
df_cda.rename(columns=essays_mapping, inplace=True)

df_cda['oilWeight'] = df_cda['oilWeight'].astype(str)

# Convert sampleDate to datetime
df_cda['sampleDate'] = pd.to_datetime(df_cda['sampleDate'], errors='coerce')

# Some extra changes
model_to_machine = {
    '789C' : 'camion',
    '789D' : 'camion',
}
# map model to machineName
df_cda['machineName'] = df_cda['machineModel'].map(model_to_machine)
df_cda['machineBrand'] = 'caterpillar'
# lowercase componentName and replace '-' with '.' in unitId
df_cda['componentName'] = df_cda['componentName'].str.lower()
df_cda['unitId'] = df_cda['unitId'].str.replace('-', '_')
# add client column
df_cda.loc[:, 'client'] = 'CDA'

df_cda.groupby(['unitId']).agg({
    'sampleDate': ['min', 'max']
})

sampleDate           
              min        max
unitId                      
T_09   2020-12-24 2026-01-07
T_10   2020-12-29 2025-12-21
T_11   2021-01-19 2025-12-27
T_12   2020-12-26 2025-12-30
T_13   2021-01-07 2025-12-21
T_14   2021-01-09 2025-12-25
T_15   2020-12-27 2026-01-10
T_16   2020-12-25 2025-12-17
T_17   2020-12-23 2026-01-01
T_18   2021-01-09 2026-01-03
T_24   2023-03-05 2026-01-02

In [6]:
valid_columns = ['client', 'sampleNumber','sampleDate', 
                'unitId', 'machineName', 'machineModel', 'machineBrand', 'machineHours', 'machineSerialNumber',
                 'componentName', 'componentHours', 'componentSerialNumber',
                 'oilMeter', 'oilBrand', 'oilType', 'oilWeight'
                 ] + list(essays['ElementNameSpanish'])

df_cda = df_cda[valid_columns].copy()
df_cda.head()

,client,sampleNumber,sampleDate,unitId,machineName,machineModel,machineBrand,machineHours,machineSerialNumber,componentName,...,Silicio,Potasio,Sodio,Zinc,Bario,Boro,Calcio,Molibdeno,Magnesio,Fósforo
1,CDA,R120-56014-0277,2026-01-07,T_09,camion,789C,caterpillar,112380.0,2BW01106,mando final trasero izquierdo,...,18.0,0.0,0.0,1105.0,0.0,0.0,3499.0,0.0,9.0,874.0
2,CDA,R120-56014-0279,2026-01-07,T_09,camion,789C,caterpillar,112380.0,2BW01106,transmision,...,7.0,0.0,0.0,1092.0,0.0,0.0,3406.0,0.0,7.0,880.0
3,CDA,R120-56014-0280,2026-01-07,T_09,camion,789C,caterpillar,112380.0,2BW01106,convertidor,...,27.0,0.0,0.0,1076.0,0.0,0.0,3241.0,0.0,8.0,862.0
4,CDA,R120-56014-0278,2026-01-07,T_09,camion,789C,caterpillar,112380.0,2BW01106,mando final trasero derecho,...,22.0,0.0,4.0,1088.0,0.0,0.0,3415.0,0.0,9.0,884.0
5,CDA,R120-56014-0282,2026-01-07,T_09,camion,789C,caterpillar,112380.0,2BW01106,sistema hidraulico,...,27.0,0.0,0.0,1067.0,0.0,0.0,3201.0,0.0,8.0,865.0


## Emin

Data of Emin comes from ALS Lab and it is stored on "data/oil/raw/emin" separated on 2 parquet files. One with the samples and other with the equipments.

In [7]:
df_emin = pd.read_parquet('raw/emin/muestrasAlsHistoricos.parquet')

df_emin.head()

,id,sampleNumber,externalCode,equipment_id,equipment_serial,equipment_tag,equipment_family_id,equipment_family_name,equipment_maker_id,equipment_maker_name,...,testResults_32_test_translation_method,testResults_32_test_testGroup_id,testResults_32_test_testGroup_name,testResults_32_test_valueType,testResults_32_test_order,testResults_32_file,program_id,program_name,equipment,compartment
0,6995177,2510035262,None,595151.0,KMTPC192KLC072317,EXC-045,4535.0,EXCAVADORA,3018.0,KOMATSU,...,None,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN
1,6995028,2510035813,None,595151.0,KMTPC192KLC072317,EXC-045,4535.0,EXCAVADORA,3018.0,KOMATSU,...,None,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN
2,6995604,2510035199,None,587288.0,1M2GR3HC7LM002096,CG-057,4552.0,CAMIÓN,2651.0,MACK,...,None,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN
3,6995289,2510035874,None,965037.0,TPL01063,BULL-038,2435.0,BULLDOZERS DE ORUGA,1938.0,CATERPILLAR,...,None,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN
4,6995189,2510035274,None,587277.0,FRS02496,CF-030,4577.0,PALA CARGADORA,1938.0,CATERPILLAR,...,None,NaN,None,None,NaN,NaN,NaN,None,NaN,NaN


Each Column with the values of the essays are not well named
> columns = sample | testElementName1 | testElementValue1 | testElementName2 | testElementValue2 | ...

Therefore, we need to map them onto a common pattern and extract the key columns.

In [8]:
# Rename main columns
df_emin.rename(columns={'equipment_tag' : 'unitId',
                        'collectionData_dateSampled' : 'sampleDate',
                        'equipment_family_name' : 'machineName',
                        'equipment_model' : 'machineModel',
                        'equipment_maker_name' : 'machineBrand',
                        'equipment_time' : 'machineHours',
                        'equipment_serial' : 'machineSerialNumber',
                        'compartment_name' : 'componentName',
                        # componentHours is not available
                        'compartment_id' : 'componentSerialNumber',
                        'collectionData_fluidTime' : 'oilMeter',
                        'collectionData_oil_manufacturer_name' : 'oilBrand',
                        'collectionData_oil_viscosity_name' : 'oilType',
                        # oilWeight is not available -> it can be inferred from oilType [future work]
                   }, inplace=True)

df_emin['componentHours'] = None
df_emin['oilWeight'] = None

name_cols = [col for col in df_emin.columns if ('_test_translation_name' in col)]
value_cols = [col for col in df_emin.columns if ('_resultValue' in col)]
unique_col = ['sampleNumber']

In [9]:
# Rename essay columns in df_emin
## Since we are facing a dataframe with the structure testElementName1 | testElementValue1 | testElementName2 | testElementValue2
## we will melt and pivot the dataframe to get the desired structure
df_names = df_emin.melt(
    id_vars=unique_col,
    value_vars=name_cols,
    value_name='testName'
)
df_names['test_number'] = df_names['variable'].str.extract(r'(\d+)')

df_values = df_emin.melt(
    id_vars=unique_col,
    value_vars=value_cols,
    value_name='testValue'
)
df_values['test_number'] = df_values['variable'].str.extract(r'(\d+)')

df_values.loc[:, 'testValue'] = df_values['testValue'].str.replace('-', '')
    
# replace ',' with '.'
df_values.loc[:, 'testValue'] = df_values['testValue'].str.replace(',', '.')

# replace <0,05 with 0
df_values.loc[:, 'testValue'] = df_values['testValue'].str.replace('<0.05', '0')
df_values.loc[:, 'testValue'] = df_values['testValue'].str.replace('>0.05', '0.1')

# Convert testValue to numeric, drop rows that cannot be converted
df_values['testValue'] = pd.to_numeric(df_values['testValue'], errors='coerce')
df_values.dropna(subset=['testValue'], inplace=True)

# Merge on equipment_serial and test_number
df = pd.merge(
    df_names[unique_col + ['test_number', 'testName']],
    df_values[unique_col + ['test_number', 'testValue']],
    on=unique_col + ['test_number'],
)

# Drop test_number as it's no longer needed
df = df.drop(columns=['test_number'])
df.dropna(inplace=True)

df.sort_values(by=['sampleNumber', 'testName'], inplace=True)
df.reset_index(drop=True, inplace=True)

df_emin = pd.merge(
    df_emin,
    df.pivot(index='sampleNumber', columns='testName', values='testValue'),
    on='sampleNumber',
    how='left'
)

df_emin.head()

,id,sampleNumber,externalCode,equipment_id,machineSerialNumber,unitId,equipment_family_id,machineName,equipment_maker_id,machineBrand,...,Sólidos disueltos totales,TAN colorimétrico,Titanio,Urea,Vanadio,Viscosidad 100°C,Viscosidad 40°C,Zinc,pH,Índice PQ
0,6995177,2510035262,None,595151.0,KMTPC192KLC072317,EXC-045,4535.0,EXCAVADORA,3018.0,KOMATSU,...,NaN,NaN,0.0,NaN,0.0,11.92,NaN,1246.0,NaN,1.0
1,6995028,2510035813,None,595151.0,KMTPC192KLC072317,EXC-045,4535.0,EXCAVADORA,3018.0,KOMATSU,...,NaN,NaN,0.0,NaN,0.0,11.75,NaN,1179.0,NaN,1.0
2,6995604,2510035199,None,587288.0,1M2GR3HC7LM002096,CG-057,4552.0,CAMIÓN,2651.0,MACK,...,NaN,NaN,0.0,NaN,0.0,12.32,NaN,848.0,NaN,1.0
3,6995289,2510035874,None,965037.0,TPL01063,BULL-038,2435.0,BULLDOZERS DE ORUGA,1938.0,CATERPILLAR,...,NaN,NaN,0.0,NaN,0.0,13.79,NaN,905.0,NaN,1.0
4,6995189,2510035274,None,587277.0,FRS02496,CF-030,4577.0,PALA CARGADORA,1938.0,CATERPILLAR,...,NaN,NaN,0.0,NaN,0.0,13.47,NaN,918.0,NaN,1.0


Names of the features are already in Spanish, so there is no mapping needed.

In [10]:
# Convert sampleDate to datetime
df_emin['sampleDate'] = pd.to_datetime(df_emin['sampleDate'], errors='coerce')

# lowercase componentName and replace '-' with '.' in unitId
df_emin['componentName'] = df_emin['componentName'].str.lower()
df_emin['unitId'] = df_emin['unitId'].str.replace('-', '_')
# add client column
df_emin.loc[:, 'client'] = 'EMIN'

df_emin = df_emin[valid_columns].copy()
df_emin.head()

,client,sampleNumber,sampleDate,unitId,machineName,machineModel,machineBrand,machineHours,machineSerialNumber,componentName,...,Silicio,Potasio,Sodio,Zinc,Bario,Boro,Calcio,Molibdeno,Magnesio,Fósforo
0,EMIN,2510035262,NaT,EXC_045,EXCAVADORA,PC450LC-8,KOMATSU,3455.0,KMTPC192KLC072317,mando final derecho,...,20.0,0.0,0.0,1246.0,0.0,0.0,1214.0,1.0,1519.0,1077.0
1,EMIN,2510035813,2025-09-05 04:00:00+00:00,EXC_045,EXCAVADORA,PC450LC-8,KOMATSU,3455.0,KMTPC192KLC072317,mando final izquierdo,...,22.0,0.0,0.0,1179.0,0.0,2.0,3500.0,1.0,156.0,973.0
2,EMIN,2510035199,2025-09-26 03:00:00+00:00,CG_057,CAMIÓN,GR64BX,MACK,4058.0,1M2GR3HC7LM002096,motor,...,9.0,0.0,1.0,848.0,0.0,80.0,1596.0,39.0,529.0,740.0
3,EMIN,2510035874,2025-09-26 03:00:00+00:00,BULL_038,BULLDOZERS DE ORUGA,D6XE,CATERPILLAR,1222.0,TPL01063,motor,...,14.0,0.0,1.0,905.0,0.0,63.0,1788.0,39.0,524.0,783.0
4,EMIN,2510035274,2025-09-08 03:00:00+00:00,CF_030,PALA CARGADORA,966L,CATERPILLAR,1021.0,FRS02496,motor,...,7.0,0.0,1.0,918.0,0.0,64.0,1821.0,38.0,489.0,805.0


### Store Pre-Processed Data

Since we were able to map the main fields for each data source we should abstract the process (using the provider (ALS or Finning) as key instrument).
At the same time, we can use a simplified layer of preprocessing to ensure that the format of every column is the same fololowing the next criteria:
* No Accents
* Lower Case
* Reduced Use Cases

In [11]:
def nameProtocol(series):
    '''
    Standardize names in a given column by normalizing unicode characters, and converting to lowercase.
    '''
    # Normalize unicode characters to NFKD, encode to ASCII bytes, decode back to UTF-8 string
    series = series.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    # Convert to lowercase
    series = series.str.lower()
    # Secure no accents
    series = series.str.replace('á', 'a').str.replace('é', 'e').str.replace('í', 'i').str.replace('ó', 'o').str.replace('ú', 'u')
    return series

def reduceCardinalityNames(series):
    '''
    Reduce the cardinality of names in a given column by mapping similar names to a common name.
    '''
    mapping = {
        'machineName' : {
            'bulldozer' : 'bulldozer',
            'pala' : 'pala',
        },
        'componentName' : {
            'mando final' : 'mando final',
            'hidraulico' : 'hidraulico',
            'refrig' : 'refrigerante',
            'aceite' : 'aceite',
            'vibrador' : 'vibrador',
            'cojinete ' : 'cojinete',
            'winche' : 'winche',
            'trasmision' : 'transmision',
            'transmision' : 'transmision',
            'tandem' : 'tandem',
            'cubo' : 'cubo',
            'eje' : 'eje',
            'engranaje' : 'engranaje',
            'freno' : 'freno',
            'retardador' : 'retardador',
            'rueda' : 'rueda',
            'direccion' : 'direccion',
            'diferencial' : 'diferencial',
        },
        'machineBrand' : {
            'cat' : 'caterpillar',
        }
    }
    
    # Get the mapping for this series based on its name
    if series.name not in mapping:
        return series
    
    series_mapping = mapping[series.name]
    
    # Create a copy to avoid SettingWithCopyWarning
    result = series.copy()
    
    for original, new in series_mapping.items():
        mask = result.str.contains(original, na=False)
        result.loc[mask] = new
        
    return result

dfs = [df_cda, df_emin]

new_dfs = []
for dfi in dfs:
    print('Initial Distribution',dfi['client'].iloc[0], dfi.shape)
    
    # Apply nameProtocol to componentName, machineName, and machineBrand
    dfi['componentName'] = nameProtocol(dfi['componentName'])
    dfi['machineName'] = nameProtocol(dfi['machineName'])
    dfi['machineBrand'] = nameProtocol(dfi['machineBrand'])


    # Apply reduceCardinalityNames to componentName, machineName, and machineBrand
    dfi['componentName'] = reduceCardinalityNames(dfi['componentName'])
    dfi['machineName'] = reduceCardinalityNames(dfi['machineName'])
    dfi['machineBrand'] = reduceCardinalityNames(dfi['machineBrand'])

    
    print('Numero de maquinas únicas', dfi['machineName'].nunique(), dfi['machineName'].unique())
    print('Numero de componentes únicos', dfi['componentName'].nunique(), dfi['componentName'].unique())
    
    new_dfs.append(dfi)
    client = dfi.client.unique()[0]

    dfi.to_parquet(f'processed/{client.lower()}.parquet')

Initial Distribution CDA (6833, 37)
Numero de maquinas únicas 1 ['camion']
Numero de componentes únicos 9 ['mando final' 'transmision' 'convertidor' 'hidraulico' 'motor'
 'direccion' 'rueda' 'diferencial' 'petroleo']
Initial Distribution EMIN (6519, 37)
Numero de maquinas únicas 16 ['excavadora' 'camion' 'bulldozer' 'pala' 'motoniveladora' 'trituradora'
 'rodillo' 'camion articulado o dumper articulado' 'planta' 'industrial'
 'no informado' 'generador' 'cargador frontal' 'soldador' 'grua' 'diesel'
 None]
Numero de componentes únicos 29 ['mando final' 'motor' 'transmision' 'hidraulico' 'direccion'
 'diferencial' 'damper' 'mando de giro' 'refrigerante'
 'mando circulo reductor' 'retardador' 'cubo' 'eje' 'cojinete' 'freno'
 'motor de giro' 'arbol de paso' 'mando rotacion'
 'mando del circulo reductor' 'tandem' 'aceite' 'engranaje'
 'sistema vibracion' 'adblue' 'vibrador' 'winche' 'reductor de giro' None
 'convertidor' 'caja de velocidades oruga']


## Stewart Limits

Now we will refine the analysis. We will compute the limit values based on percentiles.
* rules:
    * normal < 0.85
    * alerta < 0.95
    * grave < 97.5
    * urgente > 97.5

Also, this will be computed at the component level, so we can increase the knowledge of the essays.

In [12]:
import numpy as np
def stewart_limits(serie):
    """
    Cálculo de límites estadísticos para un conjunto de datos.
    Reglas:
        normal < 0.85
        alerta < 0.95
        grave < 97.5
        urgente > 97.5
    """
    serie = serie[serie != 0].dropna()
    normal = np.ceil(serie.quantile(0.9))
    alert = np.ceil(serie.quantile(0.95))
    critic = np.ceil(serie.quantile(0.98))
    
    if alert <= normal:
        alert = normal + 1
    if critic <= alert:
        critic = alert + 1

    return {
        'threshold_normal': normal,
        'threshold_alert': alert,
        'threshold_critic': critic,
    }
    
sl = {}
for dfi in new_dfs:
    client = dfi.client.unique()[0]
    
    print(f'Numero de maquinas únicas ({client})', dfi['machineName'].nunique(), dfi['machineName'].unique())

    sl[client] = {}
    for machine in dfi['machineName'].unique():
        sl[client][machine] = {}
        for component in dfi[dfi.machineName == machine].componentName.unique():
            sl[client][machine][component] = {}
            # filter the right rows and drop columns with only NaN
            aux_df = dfi[(dfi.componentName == component) & (dfi.machineName == machine)].copy().dropna(axis=1, how='all')
            for essay in list(essays['ElementNameSpanish']):
                if (essay in aux_df.columns):
                    if (aux_df[essay].nunique() > 3):
                        sl[client][machine][component][essay] = stewart_limits(aux_df.loc[:, essay])
                
# Save sl as a json file in data/ALS/processed/stewart_limits.json
import json
with open('processed/stewart_limits.json', 'w') as f:
    json.dump(sl, f, indent=4)

Numero de maquinas únicas (CDA) 1 ['camion']
Numero de maquinas únicas (EMIN) 16 ['excavadora' 'camion' 'bulldozer' 'pala' 'motoniveladora' 'trituradora'
 'rodillo' 'camion articulado o dumper articulado' 'planta' 'industrial'
 'no informado' 'generador' 'cargador frontal' 'soldador' 'grua' 'diesel'
 None]


## Comments

With the Stewart Limits Computed, we will process the data to get comments based on the AI criteria.
To do so, we need to check each essay against the limits. Based on that criteria, we will compute a column called "reportStatus" based on the following path:
1. Assign points to the essay status:
* None : 0 points
* Marginal : 1 point
* Condenatorio : 3 points
* Critic : 5 points
2. Sum the points of the essays at a report level (we will call it essaySum)
3. Apply logic over the sum to get reportStatus
* If essaySum < 3 => reportStatus = Normal
* If essaysSum < 5 => reportStatus = Alerta
* Else reportStatus = Anortmal

We later will apply an API call to OpenAI to get a recommendation over all the report that are not classified as Normal.

In [13]:
# Functions to pre process the data for comments

def identify_threshold(value, marginal, condenatorio, critico):
    """Identify which threshold has been reached based on the value. Returns the name of the threshold and its limit value."""
    if pd.isna(value):
        return None, None
    elif value >= critico:
        return 'Critico', critico
    elif value >= condenatorio:
        return 'Condenatorio', condenatorio
    elif value >= marginal:
        return 'Marginal', marginal
    else:
        return None, None
      
essayPointsDict = {
        'Critico': 5,
        'Condenatorio': 3,
        'Marginal': 1
}

statusThresholds = {
    'Anormal': 5,
    'Normal': 3
}

def getReportStatus(essays_broken:pd.DataFrame, 
                    points: dict = essayPointsDict, 
                    thresholds: dict = statusThresholds) -> str:
    """
    Determine the overall report status based on the broken essays.
    If the sum of essay points is greater than or equal to the 'Anormal' threshold, return 'Anormal'.
    If the sum of essay points is less than the 'Normal' threshold, return 'Normal'.
    Otherwise, return 'Alerta'.
    """
    if essays_broken.empty:
        return 'Normal'
    
    essays_broken.loc[:, 'essayPoints'] = essays_broken['limite transgredido'].map(points)
    essaySum = essays_broken['essayPoints'].sum()
    if essaySum < thresholds['Normal']:
        return 'Normal'
    elif essaySum >= thresholds['Anormal']:
        return 'Anormal'
    else:
        return 'Alerta'

def process_row_data_for_comment(row, limits=sl, essays_list=essays['ElementNameSpanish']):
    
    client = row['client']
    machine = row['machineName']
    component = row['componentName']
    sel_limits = limits.get(client, {}).get(machine, {}).get(component, {})
    
    essays_broken = []
    for essay in essays_list:
        
        marginal = sel_limits.get(essay, {}).get('threshold_normal', np.nan)
        condenatorio = sel_limits.get(essay, {}).get('threshold_alert', np.nan)
        critico = sel_limits.get(essay, {}).get('threshold_critic', np.nan)
        value = row[essay]
        
        # print(f'Comparing essay: {essay}, value: {value}, marginal: {marginal}')
        if pd.isna(value) or value < marginal or pd.isna(marginal):
            continue
        else:
            threshold_reached, value_lim = identify_threshold(value, marginal, condenatorio, critico)
            
            essay_details = {
                'elemento': essay,
                'valor': value,
                'limite transgredido' : threshold_reached,
                'valor_limite': value_lim
            }
            
            essays_broken.append(essay_details)
            
    essays_broken = pd.DataFrame(essays_broken)
    final_status = getReportStatus(essays_broken)
    
    return essays_broken, final_status


row = dfi.sort_values(by='Hierro').iloc[-1]
essays_issues = process_row_data_for_comment(row, limits=sl)
essays_issues

(    elemento   valor limite transgredido  valor_limite  essayPoints
 0     Hierro  2525.0             Critico         437.0            5
 1      Cromo    20.0             Critico          10.0            5
 2     Níquel     3.0            Marginal           3.0            1
 3     Estaño     6.0             Critico           5.0            5
 4  Manganeso    22.0             Critico          13.0            5
 5  Molibdeno     6.0            Marginal           6.0            1,
 'Anormal')

In [14]:
from typing import Tuple, Dict
from openai import OpenAI
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

client = OpenAI()

def create_final_prompt(example_sample : pd.Series, values_sample: pd.DataFrame) -> str:
    """
    Create prompts for LLM based on sample data from oil analysis dataframe.
    
    Args:
        example_sample (pd.Series): Series with sample data for a specific oil analysis.
        values_sample (pd.DataFrame): DataFrame with essays that exceeded limits.

    Returns:
        PROMPT (str) : Prompt for AI recommendation.
    """
    component_sample = example_sample['componentName']
    machine_sample = example_sample['machineName']
    machine_model_sample = example_sample['machineModel']

    PROMPT = f"""
    Analiza una muestra para el siguiente equipo:
    Componente: {component_sample.lower()}
    Máquina: {machine_sample.lower()} - {machine_model_sample.upper()}

    Los valores de la muestra son : 
    {values_sample}
    """
    
    return PROMPT

TEMP_DEFAULT = 0.9
MODEL = "gpt-4o-mini"

def create_recommendation(prompt: str, client) -> str:
    """
    Create a recommendation based on the analysis prompt.

    Args:
        prompt (str): The analysis prompt for the oil sample.
        client: OpenAI client instance.

    Returns:
        final_message (str): The recommendation based on the analysis prompt.
    """
    
    context = {"role": "system",
                    "content": "Eres un ingeniero mecanico, especialista en equipos mineros y debes realizar diagnosticos precisos sobre las medidas de un equipo, entregando comentarios breves respecto a los análisis de aceite realizados y recomendaciones concretas de mantención. Considera que al haber presencia de Zinc, Bario, Boro, Calcio, Molibdeno, Magnesio o Fósforo en el aceite no se debe sugerir cambio de componentes o de aceite. Tus respuestas deben ser de 150 palabras o menos."}
    messages = [context]

    # Set previous interaction to refine results
    ## Example 1
    messages.append({"role": "user", "content": 'Analiza una muestra para el siguiente equipo:\nComponente: Aceite motor Diesel 15W40\nLos valores de la muestra son:\n                    elemento  valor          limite transgredido  valor limite\n           Contenido de agua    8.3 limite superior condenatorio           0.3\nViscosidad cinemática @ 40°C  144.6 limite superior condenatorio         138.0'})
    messages.append({"role": "assistant", "content": 'Se aprecian niveles de desgaste y contaminación externa entre límites permisibles, sin embargo, se detecta contenido de agua 8,3% en volumen de muestra. Grado de viscosidad sobre límite superior condenatorio 144,6 por posibles rellenos con lubricante de ISO VG mayor.\nSe sugiere cambio de lubricante y mantener seguimiento riguroso cada 125 horas, para evidenciar alzas abruptas de sodio y potasio por eventual traspaso de refrigerante. Monitorear consumos de aceite y gases blancos en salida de escape.'})

    ## Example 2
    messages.append({"role": "user", "content": 'Analiza una muestra para el siguiente equipo:\nComponente: Motor Diesel\nMáquina: Camión Tolva\nLos valores de la muestra son:\n                     elemento  valor          limite transgredido  valor limite\n                       Fierro   31.1     limite superior marginal          30.0\n                        Cobre  267.7 limite superior condenatorio          15.0\n                       Estaño    2.9     limite superior marginal           1.5\n                      Silicio   30.7 limite superior condenatorio          17.0\n                    Índice PQ   20.0     limite superior marginal          20.0\nViscosidad cinemática @ 100°C   10.7 limite inferior condenatorio          11.0\n     Dilución por combustible    0.8 limite superior condenatorio           0.5'})
    messages.append({"role": "assistant", "content": 'Se detecta concentración de metales de desgaste por Fierro 31.1 ppm y Cobre 267.7 ppm, evidenciando posible abrasión excesiva en cojinetes y bujes de turbo. Silicio 30.7 ppm señala ingesta excesiva de polvo ambiental. Análisis fisicoquímico detecta material ferroso en suspensión Índice PQ 20.0. Se evidencia Dilución por combustible de 0.8%, detectando posible anormalidad en bomba de transferencia o sellos de inyectores, provocando una baja ligera en viscosidad en lubricante.\nSe recomienda priorizar cambio de lubricante y elementos filtrantes, evaluar presiones en sistema de lubricación y saturación temprana de filtros, mantener seguimiento riguroso cada 50 hrs.'})

    ## Example 3
    messages.append({"role": "user", "content": 'Analiza una muestra para el siguiente equipo:\nComponente: Motor Diesel Cummins\nMáquina: Camión Komatsu 730-E de Extracción\nLos valores de la muestra son:\n                elemento  valor          limite transgredido  valor limite\nDilución por combustible    2.2 limite superior condenatorio           0.3'})
    messages.append({"role": "assistant", "content": 'Niveles de desgaste y contaminaciones determinadas por espectrometría en tamaño y concentraciones admisibles para la operación, sin embargo, análisis físico químico detecta Dilución por combustible 2.2%, altamente perjudicial para la operación, evidenciando posibles anomalías en toberas o sellos de inyectores. Grado de visosidad normal en lubricante.\nSe sugiere priorizar intervención mecánica y efectuar cambio de lubricante, junto con envío de contramuestra para realizar seguimiento a deterioro en sellos/toberas de inyectores o bomba de transferencia. Evaluar presiones en sistema de lubricación y saturación temprana de filtros.'})    

    # Add user message containing the prompt
    messages.append({"role": "user", "content": prompt})

    # Call OpenAI API to generate the response based on the conversation history
    response = client.chat.completions.create(
        model = MODEL,
        temperature = TEMP_DEFAULT,
        messages = messages
    )

    final_message = response.choices[0].message.content
    
    return final_message


def orchestrate_comment(df_with_data: pd.DataFrame, 
                        identifier: str,
                        identifier_colname: str = 'sampleNumber',
                        limits: dict = sl, 
                        client = client
                        ) -> Dict:
    """
    Orchestrate the classification and comment creation process for a given identifier.
    
    PRODUCTION-READY: Returns complete report metadata including status classification.

    Args:
        df_with_data (pd.DataFrame): The DataFrame containing oil analysis data.
        identifier (str): The unique identifier for the oil analysis sample.
        identifier_colname (str): Column name for the identifier.
        limits (dict): Dictionary containing limit values for analysis.
        client: OpenAI client instance.
        
    Returns:
        Dict with keys:
            - 'reportStatus': str ('Normal', 'Alerta', 'Anormal')
            - 'severityScore': int (sum of essay points)
            - 'essaysBreached': int (count of essays that exceeded limits)
            - 'aiPrompt': str (prompt sent to AI, None if Normal)
            - 'aiRecommendation': str (AI response, None if Normal)
            - 'requiresAction': bool (True if non-Normal status)
    """
    df_sample = df_with_data[df_with_data[identifier_colname] == identifier].iloc[0]
    
    # Ensure there is data for the identifier
    if df_sample.empty:
        raise ValueError(f"No data found for identifier: {identifier}")
    
    # Check if there are any limit breaches and get status
    df_values, final_status = process_row_data_for_comment(df_sample, limits=limits)
    
    # Calculate severity metrics
    if not df_values.empty:
        essays_breached = len(df_values)
        severity_score = df_values.get('essayPoints', pd.Series([0])).sum()
    else:
        essays_breached = 0
        severity_score = 0
    
    # Initialize result dictionary
    result = {
        'reportStatus': final_status,
        'severityScore': int(severity_score),
        'essaysBreached': essays_breached,
        'requiresAction': final_status != 'Normal'
    }
    
    # Only generate AI recommendation for non-Normal reports
    if final_status == 'Normal':
        result['aiPrompt'] = None
        result['aiRecommendation'] = "No se detectan anomalías en los parámetros analizados."
    else:
        # Create prompt and get AI recommendation
        prompt = create_final_prompt(df_sample, df_values)
        ai_response = create_recommendation(prompt, client)
        
        result['aiPrompt'] = prompt
        result['aiRecommendation'] = ai_response

    return result


# Test the improved function
print("Testing improved orchestrate_comment function:\n")
examples_for_comments = {
    i : dfi.sort_values(by='Hierro').iloc[-i]['sampleNumber'] for i in range(5)
}

for state, identifier in examples_for_comments.items():
    print(f"\n{'='*80}")
    print(f"Example {state}: {identifier}")
    print('='*80)
    
    result = orchestrate_comment(dfi, identifier, limits=sl, client=client)
    
    print(f"Report Status: {result['reportStatus']}")
    print(f"Severity Score: {result['severityScore']}")
    print(f"Essays Breached: {result['essaysBreached']}")
    print(f"Requires Action: {result['requiresAction']}")
    print(f"\nAI Recommendation:\n{result['aiRecommendation']}")


Testing improved orchestrate_comment function:


Example 0: 2310011952
Report Status: Anormal
Severity Score: 8
Essays Breached: 2
Requires Action: True

AI Recommendation:
El análisis de la muestra de refrigerante indica niveles altos de Potasio 8094 ppm y Molibdeno 606 ppm, ambos valores transgredidos y considerados críticos. La presencia de Potasio sugiere contaminación por refrigerante, lo que puede afectar la eficiencia del sistema de enfriamiento. Asimismo, el Molibdeno en niveles críticos puede deberse a la degradación de componentes, como los sellos, o a la contaminación por aceite.

Se recomienda realizar un cambio inmediato de refrigerante y revisar posibles fugas en el sistema que puedan estar causando la contaminación. También es aconsejable inspeccionar el estado de los componentes asociados, como radiadores y bombas, para evitar futuras fallas. Monitorear los niveles de Potasio y Molibdeno en futuros análisis para evaluar la efectividad de las medidas implementadas.

Exam

In [15]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading
from typing import Optional, Dict
from tqdm import tqdm
import logging
from datetime import datetime

logger = logging.getLogger(__name__)

def process_single_identifier(
    identifier: str,
    df_in: pd.DataFrame,
    limits: dict,
    client
) -> Optional[Dict]:
    """
    Process a single identifier and return comprehensive classification results.
    
    PRODUCTION-READY: Captures full report state for database storage.
    
    Parameters
    ----------
    identifier : str
        Report identifier (sampleNumber)
    df_in : pd.DataFrame
        Input DataFrame with oil analysis data
    limits : Dict
        Stewart limits dictionary
    client : OpenAI client
        OpenAI client instance
        
    Returns
    -------
    Optional[Dict]
        Dictionary with complete report metadata:
        - 'sampleNumber': str
        - 'reportStatus': str (Normal/Alerta/Anormal)
        - 'severityScore': int
        - 'essaysBreached': int
        - 'requiresAction': bool
        - 'aiPrompt': str or None
        - 'aiRecommendation': str
        - 'classificationTimestamp': datetime
    """
    try:
        result = orchestrate_comment(
            df_in, 
            identifier, 
            limits=limits, 
            client=client
        )
        
        # Add identifier and timestamp for tracking
        result['sampleNumber'] = identifier
        result['classificationTimestamp'] = datetime.now()
        
        return result
        
    except ValueError as e:
        logger.warning(f"Skipping {identifier}: {e}")
        return None
    except Exception as e:
        logger.error(f"Error processing {identifier}: {e}")
        return None


MAX_WORKERS = 18

def generate_all_recommendations(
    df: pd.DataFrame,
    limits: dict,
    client,
    max_workers: int = MAX_WORKERS
) -> pd.DataFrame:
    """
    Generate classifications and AI recommendations for all reports in parallel.
    
    PRODUCTION-READY: Adds complete report state to dataframe for tracking and decision-making.
    
    This function:
    1. Classifies each report (Normal/Alerta/Anormal)
    2. Calculates severity metrics
    3. Generates AI recommendations only for non-Normal reports (optimization)
    4. Adds timestamp for auditing
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with oil analysis data (must have 'sampleNumber' column)
    limits : Dict
        Stewart limits dictionary
    client : OpenAI client
        OpenAI client instance
    max_workers : int, optional
        Maximum number of parallel workers (default: 18)
        
    Returns
    -------
    pd.DataFrame
        Input DataFrame with added columns:
        - reportStatus: Classification (Normal/Alerta/Anormal)
        - severityScore: Numeric severity (sum of essay points)
        - essaysBreached: Count of essays exceeding limits
        - requiresAction: Boolean flag for non-Normal reports
        - aiPrompt: The prompt sent to LLM (None for Normal)
        - aiRecommendation: The AI response
        - classificationTimestamp: When classification was performed
        
    Examples
    --------
    >>> df_classified = generate_all_recommendations(
    ...     df_oil_data,
    ...     stewart_limits,
    ...     openai_client
    ... )
    >>> # Query reports requiring action
    >>> urgent_reports = df_classified[df_classified['requiresAction'] == True]
    >>> # Filter by severity
    >>> critical_reports = df_classified[df_classified['severityScore'] >= 5]
    """
    
    # Get unique identifiers
    unique_identifiers = df['sampleNumber'].unique().tolist()
    logger.info(f"Processing {len(unique_identifiers)} reports")
    
    # Process in parallel with ThreadPoolExecutor
    results = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_identifier = {
            executor.submit(
                process_single_identifier,
                identifier,
                df,
                limits,
                client
            ): identifier
            for identifier in unique_identifiers
        }
        
        # Collect results with progress bar
        with tqdm(total=len(unique_identifiers), desc="Classifying reports & generating AI recommendations") as pbar:
            for future in as_completed(future_to_identifier):
                result = future.result()
                if result is not None:
                    results.append(result)
                pbar.update(1)
    
    # Create DataFrame from results
    df_classifications = pd.DataFrame(results)
    
    # Log statistics
    logger.info(f"Classification complete: {len(df_classifications)} reports processed")
    if 'reportStatus' in df_classifications.columns:
        status_counts = df_classifications['reportStatus'].value_counts()
        logger.info(f"Status distribution:\n{status_counts}")
    if 'requiresAction' in df_classifications.columns:
        action_count = df_classifications['requiresAction'].sum()
        logger.info(f"Reports requiring action: {action_count}")
    
    # Merge with original DataFrame
    df_final = pd.merge(
        df,
        df_classifications,
        on='sampleNumber',
        how='left'
    )
    
    return df_final


# Test the production-ready pipeline
print("\n" + "="*80)
print("TESTING PRODUCTION-READY CLASSIFICATION PIPELINE")
print("="*80 + "\n")

# Test with a sample of data
df_with_classifications = generate_all_recommendations(
    dfi.sort_values(by='Hierro').iloc[-40:],
    sl,
    client
)

print("\n" + "="*80)
print("CLASSIFICATION RESULTS SUMMARY")
print("="*80)
print(f"\nColumns added to dataframe:")
print(df_with_classifications[['sampleNumber', 'reportStatus', 'severityScore', 
                                'essaysBreached', 'requiresAction']].head(10))

print(f"\n\nReport Status Distribution:")
print(df_with_classifications['reportStatus'].value_counts())

print(f"\n\nReports Requiring Action:")
print(df_with_classifications['requiresAction'].value_counts())

print(f"\n\nSample of reports requiring action:")
action_required = df_with_classifications[df_with_classifications['requiresAction'] == True]
if not action_required.empty:
    for idx, row in action_required.head(3).iterrows():
        print(f"\n{'-'*80}")
        print(f"Sample: {row['sampleNumber']}")
        print(f"Status: {row['reportStatus']} | Severity: {row['severityScore']} | Essays Breached: {row['essaysBreached']}")
        print(f"Recommendation:\n{row['aiRecommendation']}")



TESTING PRODUCTION-READY CLASSIFICATION PIPELINE



Classifying reports & generating AI recommendations: 100%|██████████| 40/40 [00:10<00:00,  3.72it/s]


CLASSIFICATION RESULTS SUMMARY

Columns added to dataframe:
  sampleNumber reportStatus  severityScore  essaysBreached  requiresAction
0   2310021432      Anormal              5               3            True
1   2310052306      Anormal             50              12            True
2   2510035302       Normal              0               0           False
3   2510073767      Anormal             12               6            True
4   2310029599      Anormal             10               4            True
5   2310044467      Anormal             12               4            True
6   2310052248       Normal              1               1           False
7   2410045077       Alerta              3               3            True
8   2310051385      Anormal             15               3            True
9   2310010174      Anormal             10               2            True


Report Status Distribution:
reportStatus
Anormal    32
Normal      4
Alerta      4
Name: count, dtype: int64


R

In [17]:
df_with_classifications.head()

,client,sampleNumber,sampleDate,unitId,machineName,machineModel,machineBrand,machineHours,machineSerialNumber,componentName,...,Molibdeno,Magnesio,Fósforo,reportStatus,severityScore,essaysBreached,requiresAction,aiPrompt,aiRecommendation,classificationTimestamp
0,EMIN,2310021432,2023-07-21 00:00:00+00:00,CF_031,pala,972L,caterpillar,294.0,M7P00269,diferencial,...,0.0,10.0,885.0,Anormal,5,3,True,\n Analiza una muestra para el siguiente eq...,Los resultados del análisis del aceite del dif...,2026-01-27 11:25:14.882296
1,EMIN,2310052306,2024-05-02 00:00:00+00:00,EXC_051,excavadora,PC450LC-8,komatsu,4744.0,72546,hidraulico,...,27.0,86.0,593.0,Anormal,50,12,True,\n Analiza una muestra para el siguiente eq...,"Los resultados del análisis son alarmantes, co...",2026-01-27 11:25:14.712998
2,EMIN,2510035302,2025-10-19 03:00:00+00:00,EXC_051,excavadora,PC450LC-8,komatsu,4744.0,72546,transmision,...,24.0,315.0,887.0,Normal,0,0,False,None,No se detectan anomalías en los parámetros ana...,2026-01-27 11:25:10.636646
3,EMIN,2510073767,2025-09-29 03:00:00+00:00,EXC_056,excavadora,350,caterpillar,519.0,CAT00350LYDW20076,mando final,...,1.0,18.0,1144.0,Anormal,12,6,True,\n Analiza una muestra para el siguiente eq...,Los resultados muestran niveles de metal de de...,2026-01-27 11:25:15.448401
4,EMIN,2310029599,2023-08-16 00:00:00+00:00,EXC_048,excavadora,320,caterpillar,908.0,ZBN31436,mando final,...,0.0,14.0,1019.0,Anormal,10,4,True,\n Analiza una muestra para el siguiente eq...,Los análisis de aceite del mando final de la e...,2026-01-27 11:25:14.954741


## Evaluate Machine Status

One of the main concerns of the clients is get to know "how the machines are right now". To get that info, we need to get the latest reports per machine/component and be able to summarize the info.

To do that, we will map the reportStatus to a numericReportStatus and compute the sum of them onto the classification of the Machine.

Steps:
1. Get the latest report per ID
2. Map the status of the report to numeric value (numericReportStatus)
* Anormal -> 2
* Alerta -> 1
* Normal -> 0
3. Compute the sum as totalNumericStatus
4. Classify the totalNumericStatus onto:
* "Normal" If totalNumericStatus <2
* "Alerta" If totalNumericStatus <4
* "Anormal" else

In [21]:
reportPointsDict = {
        'Anormal': 2,
        'Alerta': 1,
        'Normal': 0
}

machineStatusThresholds = {
    'Anormal': 4,
    'Normal': 2
}

def classify_machine_status(total_score: int,
                            thresholds: dict = machineStatusThresholds) -> str:
    """
    Classify machine status based on total score and thresholds.
    """
    if total_score >= thresholds['Anormal']:
        return 'Anormal'
    elif total_score < thresholds['Normal']:
        return 'Normal'
    else:
        return 'Alerta'

def get_machine_status(df_in : pd.DataFrame, 
                       date_col : str ='sampleDate', 
                       unit_col : str ='unitId', 
                       component_col : str ='componentName',
                       points_dict : dict = reportPointsDict,
                       threshold_dict : dict = machineStatusThresholds,
                       apply_threshold : bool =False,
                       threshold_days : int =180):
    """
    Get the status of machines based on the last sample date.
    
    Parameters
    ----------
    df_in : pd.DataFrame
        Input DataFrame with machine data.
    date_col : str
        Column name for sample date.
    unit_col : str
        Column name for unit identifier.
    component_col : str
        Column name for component identifier.
    apply_threshold : bool
        Whether to apply the inactivity threshold.
    threshold_days : int
        Threshold in days to consider a machine as 'Inactive'.
    """
    df_in = df_in.copy() if not apply_threshold else df_in[df_in[date_col] >= (pd.Timestamp.now() - pd.Timedelta(days=threshold_days))]
    last_essays = df_in.groupby([unit_col, component_col])[date_col].max().reset_index()
    
    df_last = pd.merge(df_in,
                   last_essays, 
                   on=[unit_col, component_col, date_col], 
                   how='right')
    
    df_last.loc[:, 'numericReportStatus'] = df_last['reportStatus'].map(points_dict)
    
    totalScores = df_last.groupby([unit_col], as_index=False)['numericReportStatus'].sum()
    totalScores.rename(columns={'numericReportStatus' : 'totalNumericStatus'}, inplace=True)
    
    totalScores.loc[:, 'machineStatus'] = totalScores['totalNumericStatus'].apply(
        lambda x: classify_machine_status(x, thresholds=threshold_dict)
    )
    
    return totalScores

# Process CDA data
df_example = get_machine_status(df_with_classifications)
df_example.head()

,unitId,totalNumericStatus,machineStatus
0,CF_031,8,Anormal
1,CG_063,4,Anormal
2,CP_004,2,Alerta
3,CT_162,2,Alerta
4,CT_169,2,Alerta


## Final Filtering

In [22]:

def filterInvalidSamples(df_in):
    '''
    Filter out samples with invalid or missing critical information.
    '''
    df_out = df_in.copy()
    # filter those machineName that has less than 100 samples
    valid_machines = df_out['machineName'].value_counts()
    valid_machines = valid_machines[valid_machines >= 100].index.tolist()
    df_out = df_out[df_out['machineName'].isin(valid_machines)]
    
    # filter those componentName that has less than 20 samples
    valid_components = df_out['componentName'].value_counts()
    valid_components = valid_components[valid_components >= 100].index.tolist()
    df_out = df_out[df_out['componentName'].isin(valid_components)]
    
    return df_out
dfi = filterInvalidSamples(dfi)
print('Final Distribution',dfi['client'].iloc[0], dfi.shape)


Final Distribution EMIN (6069, 37)


## Dashboard

Finally the idea is to show the results using Dash and Plotly.

To do that we will separate the visualizations onto 3 sections (or tabs) in the next cells we will write the code of the charts and structure of each tab.

The tabs are defines as follows:


1. 📊Limit Visualization:
 Limits that triggers an alert
2. 🚜Machine Status:
 Status of the different equipments based on latest available data
3. 📝Report Detail:
 Detail of a report, with the status tag, the comment generated by the AI and the visualization of the essays itself (the values of the essays)

 - Important Note : The data will be pre-filtered in order to secure that there is no leaking between clients on what data is concerned.

### 📊 Limit Visualization

We will only have a table to visualize the limits of the essays corresponding to an individual pair of machine, component.
Therefore we must implement a filter to select the desired pair of machine, component

In [25]:
# we will use emin data as an example
limits_data = sl['EMIN'].copy()

import plotly.graph_objects as go
import plotly.express as px

# Build the structure: machine -> component -> essays
machine_options = list(limits_data.keys())
all_essays = list(essays['ElementNameSpanish'])

fig = go.Figure()

# Create all possible table traces for each machine-component combination
trace_index = 0
trace_map = {}  # Maps (machine, component) to trace index

for machine in machine_options:
    component_options = list(limits_data[machine].keys())
    for component in component_options:
        # Get the limits for this machine-component pair
        component_limits = limits_data[machine][component]
        if component_limits:
            df_component = pd.DataFrame(component_limits).T
            df_component.index.name = 'Essay'
            
            # Add trace for this machine-component pair
            fig.add_trace(go.Table(
                header=dict(
                    values=['Essay'] + list(df_component.columns),
                    fill_color='paleturquoise',
                    align='left',
                    font=dict(size=12, color='black')
                ),
                cells=dict(
                    values=[df_component.index.tolist()] + [df_component[col].tolist() for col in df_component.columns],
                    fill_color='lavender',
                    align='left',
                    font=dict(size=11)
                ),
                visible=(trace_index == 0)  # Only show the first one initially
            ))
            
            trace_map[(machine, component)] = trace_index
            trace_index += 1

# Create dropdown buttons for machine selection
machine_buttons = []
for machine in machine_options:
    # Get all components for this machine
    component_options = list(limits_data[machine].keys())
    # Find the first valid component
    first_component = component_options[0] if component_options else None
    
    # Create visibility array: show only traces for this machine's first component
    visibility = [False] * trace_index
    if first_component and (machine, first_component) in trace_map:
        visibility[trace_map[(machine, first_component)]] = True
    
    machine_buttons.append(
        dict(
            label=machine,
            method="update",
            args=[
                {"visible": visibility},
                {"title": f"Stewart Limits - Machine: {machine} - Component: {first_component}"}
            ]
        )
    )

# Create dropdown buttons for component selection (will be updated based on machine)
# Start with components from the first machine
first_machine = machine_options[0]
component_buttons = []
for component in limits_data[first_machine].keys():
    visibility = [False] * trace_index
    if (first_machine, component) in trace_map:
        visibility[trace_map[(first_machine, component)]] = True
    
    component_buttons.append(
        dict(
            label=component,
            method="update",
            args=[
                {"visible": visibility},
                {"title": f"Stewart Limits - Machine: {first_machine} - Component: {component}"}
            ]
        )
    )

# Create buttons for essay filtering
essay_filter_buttons = [
    dict(
        label="All Essays",
        method="restyle",
        args=[{"cells.values": [[df_component.index.tolist()] + [df_component[col].tolist() for col in df_component.columns]]}]
    )
]

# Update layout with dropdown menus
fig.update_layout(
    updatemenus=[
        # Machine selector
        dict(
            buttons=machine_buttons,
            direction="down",
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.25,
            yanchor="top",
            bgcolor="lightblue",
            bordercolor="darkblue",
            font=dict(size=11)
        ),
        # Component selector
        dict(
            buttons=component_buttons,
            direction="down",
            showactive=True,
            x=0.3,
            xanchor="left",
            y=1.25,
            yanchor="top",
            bgcolor="lightgreen",
            bordercolor="darkgreen",
            font=dict(size=11)
        )
    ],
    title=dict(
        text=f"Stewart Limits - Machine: {first_machine} - Component: {list(limits_data[first_machine].keys())[0]}",
        font=dict(size=14)
    ),
    height=600,
    annotations=[
        dict(text="Select Machine:", x=0.1, xref="paper", y=1.28, yref="paper", 
             align="left", showarrow=False, font=dict(size=12)),
        dict(text="Select Component:", x=0.3, xref="paper", y=1.28, yref="paper", 
             align="left", showarrow=False, font=dict(size=12))
    ]
)

fig.show()

# Note: For full interactivity where component dropdown updates based on machine selection,
# you would need to use a Dash app or add custom JavaScript callbacks

### 🚜Machine Status:


This tab is more packed with charts.  The core idea is to give ther user a better understanding of "how is my fleet performing". And to enable that we will provide him with a description on the number of equipments the latest data available and the reports that justify any incidence.


It should have the following sections:
1. 📊 Distribución del Estado General por Máquina : Summary of the distribution of states at a machine lever. Pie chart with 3 states showing the % for each [normal, alerta, anormal]. Alongside a table with the priorities to check. (For prioritize we will use totalNumericStatus)
2. 🎯 Detalle Estado General por Máquina : Summary of each machine -> table with unitId | machineType | machineStatus | numberComponents | lastSampleDate | Summary
[Where summary is something like "C component(s) in state S : [C1 - ... - CX]"]
3. 📈 Distribución de Estados por Componente : Summary of the distribution of states at a report level. Pie chart with 3 states showing the % for each [normal, alerta, anormal]. Alongside a histogram showing the number of reports assigned to each component and the bar should be splitted depending on reportState
[Key difference : on section 1 we have "of 100 machines how many are on each state" on section 3 we have "of 100 reports how many are on each state"]
4. 🎯 Detalle Estado General por Componente : Summary of each component -> table with unitId | componentName | lastSampleDate | AI Recommendation


### 📝Report Detail:


In here we will explore the actual distribution of a report (what are the values of each essay, what is our assesment and recomendation)

I imagine a tab following this sections:
1. 📊 Análisis del Reporte:
Section where we visualize the values agains the condenatory limits. We will use a radar chart (polar axis) showing the values against a limit. We will stablish the 3 limits based on the machie, component of the visualization.
To make this visualization easier I figured that the best way to go is to split the radar charts based on the availability of the axis (first we secure that there is a value per each essay) and also split the radars based on the GroupElement they are onto. Basically showing one radar per groupElement and in each radar the essays that are aprt of that group.
Below the radars we should have a table with the raw values. Basically a table with essay | value | thresholds
2. 🤖 Recomendaciones de Mantenimiento
Section with text using the AI Recomendation
3. 📈 Evolución Temporal
Section with a timeseries showing each essay and how the evolved through time. (Comparing with previous reports of the same unitId)
4. 🔄 Comparación con Reportes Anteriores
A deep analysis comparing the current essay with a previous one (if available) including change in percentage and absolute value.